# Finite Difference Methods - Coefficient Calculations

>This notebook documents a method used to calculate the coefficients for a finite difference scheme of order $n$

## Description of Scheme

Consider the sketch shown below:

![Sketch of Discrete Domain](StencilSketch.png)

Here we have broken the continuous domain of some function $u$ into seven discrete points.  The points $x_j$ are a fixed distance $\delta$ apart and we denote the function $u$ being evaluated at the $j$th point $u_n$.  Imagine, we are interested in calculating the derivative of $u$ at the point $j=i$,that is, we seek $\frac{\partial u}{\partial x}\big|_{x=x_i}$.  The first step in accomplishing this is to write out the Taylor expansion centered at some arbitrary point $x=a$.

$$u(x) = u(x_i) + \frac{du}{dx}\bigg|_{a}\frac{(x-a)}{\delta} + \frac{d^2u}{dx^2}\bigg|_{a}\frac{(x-a)^2}{2!\delta^2} + \frac{d^3u}{dx^3}\bigg|_{a}\frac{(x-a)^3}{3!\delta^3} + ... + \frac{d^nu}{dx^n}\bigg|_{a}\frac{(x-a)^n}{n!\delta^n}$$

Clearly though, this equation is of limited use because we only know the _functional value_ at any given point; we do not know the derivative - that is what we wish to solve for.  We could procede by truncating the series for all terms with derivatives of order > 1 and then solve directly for the $\frac{du}{dx}\big
|_{a}$ term; however, this would give us only a second order approximation (since all truncated terms are of $x^2$ or greater).  It would be desirable to develop a more general method for approximating the derivative to any order.  

Let's proceed by rewriting the Taylor expansion replacing the derivative terms with a set of constants.

$$u(x) =\alpha_0 + \alpha_1(x-a) + \alpha_2(x-a)^2 + \alpha_3(x-a)^3 + ... + \alpha_n(x-a)^n$$

This equation states that $u(x)$ can be expressed as an $n$th order polynomial.  Let's assume that we wish to _approximate_ $u$ using an $k$th order polynomial by truncating all terms of order greater than $x^k$.  Hence, we obtain:

$$u(x) \approx \bar{u}(x) = \alpha_0 + \alpha_1(x-a) + \alpha_2(x-a)^2 + \alpha_3(x-a)^3 + ... + \alpha_k(x-a)^k$$

We can solve the constants using the following steps:
1. Select $r=k+1$ different points in the domain near $x=a$
2. For each selected point, substitute the corresponding pair $(x_j,u_j)$ into the polynomial
3. Solve the resulting system of equations to obtain each constant.

The goal here is to obtain a polynomial that locally approximates the actual function near the point $x=a$, and then use this polynomial to approximate the derivative of the function.  Specifically, we want to approximate the function _near_ $x=x_i$; so, for example if we wish to calculate a second order polynomial we would have 

$$\bar{u}(x)=\alpha_0 + \alpha_1(x-x_i)+\alpha_2(x-x_i)^2$$  
To find the constants, we could select the following points to plug into the equation:

$$\{(u_{i-2},x_{i-2}),(u_{i-1},x_{i-1}),(u_i,x_i)\}$$

Note that any deviation from $x_i$ can be expressed in terms of $\delta$. For instance, we can write $x_{i-1} = x_i - \delta$.  Thus, substituting in the three points expressed above, we obtain the following sytem of equations:

$$\begin{align} \bar{u}(x_{i-2}) &= \alpha_1 + \alpha_2(x_{i} - 2\delta - x_{i}) + \alpha_3(x_{i} - 2\delta - x_{i})^2 \\ \bar{u}(x_{i-1}) &= \alpha_1 + \alpha_2(x_{i} - \delta - x_{i})+\alpha_3(x_{i} - \delta - x_{i})^2 \\ \bar{u}(x_{i}) &= \alpha_1 + \alpha_2(x_{i} - x_{i})+\alpha_3(x_{i} - x_{i})^2 \end{align} $$

Which simplifies to:

$$\begin{align} \bar{u}_{i-2} &= \alpha_1 - 2\alpha_2\delta+4\alpha_3\delta^2 \\ \bar{u}_{i-1} &= \alpha_1 - \alpha_2\delta+\alpha_3\delta^2 \\ \bar{u}_i &= \alpha_1 \end{align}$$

Solving this system gives us the following results:

$$\begin{align} \alpha_0&=u_i \\ \alpha_1&= \frac{3u_i - 4u_{i-1} + u_{i-2}}{2\delta} \\
\alpha_2 &= \frac{u_i - 2u_{i-1} + u_{i-2}}{2\delta^2} \end{align}$$

Which we can substitute into $\bar{u}$ - note that the value of $x_i$ is irrelevent because it cancels out for all calculations, thus to simplify the math we take $x_i = 0$ to obtain:

$$\bar{u}(x) = \big(1+ \frac{3}{2\delta}x + \frac{1}{2\delta^2}x^2\big)u_i -\big(\frac{2}{\delta}x + \frac{1}{\delta^2}x^2\big)u_{i-1} + \big(\frac{1}{2\delta}x +\frac{1}{2\delta^2}x^2 \big)u_{i-2} $$

Thus, we have a second order polynomial that locally approximates $u(x)$ near $x=x_i$.  In order to use this polynomial to calculate the derivative, we want to find the function value at the "half points" of the stencil, that is, we seek $\big\{u_{i+\frac{1}{2}},u_{i-\frac{1}{2}},u_{i-\frac{3}{2}}\big\}$.  These are found just by plugging in the appropriate $x$ value into $\bar{u}$.  After some algebra, we obtain:

$$\boxed{\begin{align} u_{i+\frac{1}{2}} \approx \bar{u}\big(\tfrac{1}{2}\delta\big) &= \frac{15}{8}u_i - \frac{5}{4}u_{i-1} +\frac{3}{8}u_{i-2} \\ u_{i-\frac{1}{2}} \approx \bar{u}\big(-\tfrac{1}{2}\delta\big) &= \frac{3}{8}u_i + \frac{3}{4}u_{i-1} - \frac{1}{8}u_{i-2} \\ u_{i-\frac{3}{2}} \approx \bar{u}\big(-\tfrac{3}{2}\delta\big) &= -\frac{1}{8}u_i + \frac{3}{4}u_{i-1} + \frac{3}{8}u_{i-2}  \end{align}}$$


Clearly, this is a rather tedious process, so it would be of interest to automate the determination of the proper polynomial coefficients given the desired order $k$.  I performed the hand calculations through a third order polynomial ($O(4)$). The Results are tabulated below:

|**Order**|$u_{n}$|$k_s=0$|$k_s=1$|$k_s=2$|$k_s=3$|
|---------|-----|---------------|---------------|---------------|----------------|
|  **4**  |$u_{i-3}$|$\frac{5}{16}$ |$-\frac{1}{16}$|$\frac{1}{16}$ |$-\frac{5}{16}$ | 
|    -    |$u_{i-2}$|$\frac{15}{16}$|$\frac{9}{16}$ |$-\frac{5}{16}$|$\frac{21}{16}$ |
|    -    |$u_{i-1}$|$-\frac{5}{16}$|$\frac{9}{16}$ |$\frac{15}{16}$|$-\frac{35}{16}$|
|    -    |$u_{i}  $|$\frac{1}{16}$ |$-\frac{1}{16}$|$\frac{5}{16}$ |$\frac{35}{16}$ |
|  **3**  |$u_{i-2}$|$\frac{3}{8}$  |$-\frac{1}{8}$ |$\frac{3}{8}$  |
|    -    |$u_{i-1}$|$\frac{3}{4}$  |$\frac{3}{4}$  |$-\frac{5}{4}$ |
|    -    |$u_{i}$  |$-\frac{1}{8}$ |$\frac{3}{8}$  |$\frac{15}{8}$ |
|  **2**  |$u_{i-1}$|$\frac{1}{2}$  |$-\frac{1}{2}$ |
|    -    |$u_{i}$  |$\frac{1}{2}$  |$\frac{3}{2}$  |

|**Order**|**l**|$k_s=0$|$k_s=1$|$k_s=2$|$k_s=3$|
|---------|-----|---------------|---------------|---------------|----------------|
|  **4**  |**0**|$\frac{5}{16}$ |$-\frac{1}{16}$|$\frac{1}{16}$ |$-\frac{5}{16}$ | 
|    -    |**1**|$\frac{15}{16}$|$\frac{9}{16}$ |$-\frac{5}{16}$|$\frac{21}{16}$ |
|    -    |**2**|$-\frac{5}{16}$|$\frac{9}{16}$ |$\frac{15}{16}$|$-\frac{35}{16}$|
|    -    |**3**|$\frac{1}{16}$ |$-\frac{1}{16}$|$\frac{5}{16}$ |$\frac{35}{16}$ |
|  **3**  |**0**|$\frac{3}{8}$  |$-\frac{1}{8}$ |$\frac{3}{8}$  |
|    -    |**1**|$\frac{3}{4}$  |$\frac{3}{4}$  |$-\frac{5}{4}$ |
|    -    |**2**|$-\frac{1}{8}$ |$\frac{3}{8}$  |$\frac{15}{8}$ |
|  **2**  |**0**|$\frac{1}{2}$  |$-\frac{1}{2}$ |
|    -    |**1**|$\frac{1}{2}$  |$\frac{3}{2}$  |

## Discussion with Dr. Shahbazi
---
As I had mentioned earlier, we have the case where we know a certain function on a domain $f(x)$ and we wish to differentiate this function over this interval.  There are many ways we could proceed, but our primary goal is to reconstruct a polynomial approximation of $f(x)$ locally and then use this reconstruction to evaluate the derivative.  We will call this polynomial approximation $P(x)$ - to find an $r$th order polynomial $P(x)$, we need $r+1$ points


Considering the error term, it is evident that a local reconstruction of the polynomial is preferred to a global reconstruction because we can adjust.


Interpolation gives larger error near boundaries. Because the termms grow larger and larger

We want our stencil to be as symetric as possible.

Reconstruction refers to rebuilding a continuous function given a dataset.

1. Find table of $a_{r,k_s,l}$ values for $r \in [0,10]$ and $K_s \in [0,10]$
2. Consider a smooth function $f(x)=sin(2\pi x)$ for $x\in [0,1]$. Approximate $\frac{df}{dx}$. Find and report the error for increasing order - refining gridpoint
3. Consider a smooth function that has a discontinuous or (nearly) discontinuous derivative - show error with increasing order.   


Report the $L2$ Norm for the error, that is if the absolute error vector is just the difference between our approximation and the actual value at each grid point.  The error is then just he magnitude of this vecgtor 


In [2]:
import numpy as np

18